In [3]:
import pandas as pd
import numpy as np

In [4]:
df_price = (
    pd.read_excel("./../../data/Preise.xlsx") #./../../data/Preise.xlsx   - Pfad für Sandbox
        .rename(columns={"Jahr": "year",
                         "preis": "price"})
        .assign(year=lambda _data: pd.to_numeric(_data["year"].str.strip('"')))
        .replace([20022, "Tablet Basics"],[2022, "Tablet Basic"])
)

In [5]:
df_dt = (
    pd.read_excel("./../../data/Downtime.xlsx")
        .rename(columns={"Unnamed: 0": "year",
                         "Region A": "a",
                         "Region B": "b",
                         "Region C": "c",
                         "Region D": "d"})
)

In [6]:
df_crm = (
    pd.read_excel("./../../data/CRM_data.xlsx")
        .rename(columns={"id": "transaction_id",
                         "vorname": "firstname",
                         "nachname": "lastname"})
)

In [7]:
def read_regio_data(region, kind): 
    return (pd.read_excel(f"./../../data/Region{region}_{kind}.xlsx")
                .rename(columns={"Tablet Basic": "basic",
                                "Tablet Mini": "mini",
                                "Tablet Pro": "pro",
                                "kosten_mini": "cost_mini",
                                "kosten_basic": "cost_basic",
                                "kosten_pro": "cost_pro",
                                "PRODUKT": "product"})
                .assign(year=lambda _data: _data["date"].dt.year,
                        month=lambda _data: _data["date"].dt.month,
                        week=lambda _data: _data["date"].dt.isocalendar().week,
                        cost_basic=lambda _data: np.round(_data["cost_basic"], 2),
                        cost_mini=lambda _data: np.round(_data["cost_mini"], 2),
                        cost_pro=lambda _data: np.round(_data["cost_pro"], 2),
                        cost_per_unit=lambda _data: _data["cost_basic"].fillna(0) + _data["cost_mini"].fillna(0) + _data["cost_pro"].fillna(0),
                        cost=lambda _data: _data["cost_per_unit"] * _data["amount"]
                        )
                .merge(df_price, on=["year", "product"])
                .assign(revenue=lambda _data: _data["price"] * _data["amount"],
                        profit_per_unit=lambda _data: _data["price"] - _data["cost_per_unit"],
                        profit=lambda _data: _data["profit_per_unit"] * _data["amount"])
                .merge(df_crm[["customer_rating", "transaction_id"]], on="transaction_id"))

df_regAo = read_regio_data("A","online")
df_regAp = read_regio_data("A","phone")
df_regAs = read_regio_data("A","store")
df_regBo = read_regio_data("B","online")
df_regBp = read_regio_data("B","phone")
df_regBs = read_regio_data("B","store")
df_regCo = read_regio_data("C","online")
df_regCp = read_regio_data("C","phone")
df_regCs = read_regio_data("C","store")
df_regDo = read_regio_data("D","online")
df_regDp = read_regio_data("D","phone")
df_regDs = read_regio_data("D","store")

In [8]:
df_regAo

,date,product,discount,amount,basic,mini,pro,cost_basic,cost_mini,cost_pro,...,year,month,week,cost_per_unit,cost,price,revenue,profit_per_unit,profit,customer_rating
0,2021-11-01,Tablet Basic,1.0,5,1,0,0,170.02,NaN,NaN,...,2021,11,44,170.02,850.10,529,2645,358.98,1794.90,7.1
1,2021-11-01,Tablet Basic,1.0,7,1,0,0,170.02,NaN,NaN,...,2021,11,44,170.02,1190.14,529,3703,358.98,2512.86,5.9
2,2021-11-01,Tablet Basic,0.9,7,1,0,0,210.03,NaN,NaN,...,2021,11,44,210.03,1470.21,529,3703,318.97,2232.79,7.5
3,2021-11-01,Tablet Basic,1.0,10,1,0,0,170.08,NaN,NaN,...,2021,11,44,170.08,1700.80,529,5290,358.92,3589.20,6.6
4,2021-11-01,Tablet Basic,1.0,4,1,0,0,170.09,NaN,NaN,...,2021,11,44,170.09,680.36,529,2116,358.91,1435.64,7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8525,2022-10-31,Tablet Mini,1.0,4,0,1,0,NaN,136.09,NaN,...,2022,10,44,136.09,544.36,550,2200,413.91,1655.64,8.3
8526,2022-10-31,Tablet Mini,1.0,8,0,1,0,NaN,136.07,NaN,...,2022,10,44,136.07,1088.56,550,4400,413.93,3311.44,8.2
8527,2022-10-31,Tablet Mini,1.0,3,0,1,0,NaN,136.07,NaN,...,2022,10,44,136.07,408.21,550,1650,413.93,1241.79,6.4
8528,2022-10-31,Tablet Mini,1.0,2,0,1,0,NaN,136.06,NaN,...,2022,10,44,136.06,272.12,550,1100,413.94,827.88,7.5


In [14]:
df_regAo.describe()

,date,discount,amount,basic,mini,pro,cost_basic,cost_mini,cost_pro,transaction_id,year,month,week,cost_per_unit,cost,price,revenue,profit_per_unit,profit,customer_rating
count,8530,8530.000000,8530.000000,8530.000000,8530.000000,8530.000000,4771.000000,2220.000000,1539.000000,8530.000000,8530.000000,8530.000000,8530.0,8530.000000,8530.000000,8530.000000,8530.000000,8530.000000,8530.000000,8530.000000
mean,2022-04-12 17:14:09.988276736,0.979824,5.949472,0.559320,0.260258,0.180422,259.701626,173.370869,799.264698,142478.594959,2021.789918,6.417351,26.133998,334.582434,1965.807559,612.656506,3607.648300,278.074073,1641.840741,6.709226
min,2021-11-01 00:00:00,0.900000,1.000000,0.000000,0.000000,0.000000,150.150000,136.050000,566.330000,22.000000,2021.000000,1.000000,1.0,136.050000,137.910000,499.000000,499.000000,-49.410000,-691.740000,2.100000
25%,2022-01-12 00:00:00,1.000000,4.000000,0.000000,0.000000,0.000000,200.390000,156.377500,756.530000,58411.500000,2022.000000,3.000000,12.0,183.002500,886.342500,499.000000,2116.000000,198.205000,856.207500,5.400000
50%,2022-04-04 00:00:00,1.000000,6.000000,1.000000,0.000000,0.000000,247.060000,174.320000,806.140000,129920.500000,2022.000000,6.000000,25.0,233.975000,1450.390000,529.000000,3400.000000,277.540000,1474.325000,6.700000
75%,2022-07-06 00:00:00,1.000000,8.000000,1.000000,1.000000,0.000000,308.080000,190.110000,848.885000,218503.750000,2022.000000,10.000000,41.0,352.142500,2398.230000,550.000000,4491.000000,364.860000,2245.660000,8.000000
max,2022-10-31 00:00:00,1.000000,17.000000,1.000000,1.000000,1.000000,548.410000,214.680000,894.660000,339990.000000,2022.000000,12.000000,52.0,894.660000,9557.520000,999.000000,12987.000000,483.950000,7110.600000,10.000000
std,NaN,0.040134,2.755764,0.496498,0.438801,0.384561,73.025714,19.952004,60.758676,95505.108531,0.407391,3.634301,15.83581,229.381698,1645.522463,185.087746,1959.817049,100.716255,995.418715,1.744655
